<a href="https://colab.research.google.com/github/mustafamunir2003/GEN-AI/blob/main/WhisperAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio numpy transformers

In [ ]:
!pip install -q gradio torch transformers datasets safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# Check for GPU support
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load Whisper Large V3 model
model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

# Create ASR pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# Transcription function
def transcribe_audio(audio):
    """
    Processes an audio file and returns the transcribed text.
    """
    if not audio:
        return "No audio received. Please try again!"

    result = pipe(audio)
    return result["text"]

# Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown(
        """
        # 🎙 **Whisper Large V3 - Real-Time Speech-to-Text**
        > Experience state-of-the-art speech recognition with OpenAI's Whisper Large V3.
        🎧 Speak into the microphone, and watch as your words are transcribed instantly!
        """,
        elem_id="title",
    )

    with gr.Row():
        audio_input = gr.Audio(sources="microphone", type="filepath", label="🎤 Speak Now")
        output_text = gr.Textbox(label="📝 Transcription", interactive=False, lines=3)

    transcribe_btn = gr.Button("⏳ Transcribe", variant="primary")
    transcribe_btn.click(transcribe_audio, inputs=[audio_input], outputs=[output_text])

    gr.Markdown("#### 🚀 Powered by OpenAI Whisper Large V3 & Gradio")

# Launch the app
app.launch()


Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d4a128417ae66f872e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
